# modeling

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df 
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  confusion_matrix,recall_score,precision_score,accuracy_score
from sklearn.utils import resample
from sklearn.feature_selection import SelectFromModel 
from sklearn.model_selection import GridSearchCV 
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier  

from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN
import time
from imblearn.metrics import geometric_mean_score

import warnings
warnings.filterwarnings(action='ignore')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  w

In [2]:
def get_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc = roc_auc_score(y_test, pred)
    g_mean = geometric_mean_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:4f}, 정밀도 {1:4f}, 재현율 {2:4f}, AUC {3:4f},G_MEAN {4:4f}'.format(accuracy , precision , recall,roc,g_mean))
    

def get_model_train_eval(model,ftr_train = None, ftr_test = None, tgt_train = None, tgt_test = None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)

In [3]:
df =pd.read_csv('C:/Users/koyounghun/Desktop/찌르레기/Loan_data/Loan_train.csv')
df = df.drop('Id',axis = 1)


from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()

data = df


for e in data.columns:
    if data[e].dtype == 'object':
        labelEncoder.fit(list(data[e].values))
        data[e] = labelEncoder.transform(data[e].values)
        
        # Accommodate the data that has been changed
        df = data
        
y = df.Risk_Flag
X = df.drop('Risk_Flag', axis=1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# smote

In [6]:
start = time.time()


smote = SMOTE(random_state = 101)

X_smote, y_smote = smote.fit_resample(X_train, y_train)
end = time.time()

print(f"{end - start:.5f} sec")

1.10499 sec


a , b = X_smote, y_smote
a = pd.DataFrame(a)
b = pd.DataFrame(b)

a.to_csv('C:/Users/koyounghun/Desktop/찌르레기/Loan_data/x_smote.csv')
b.to_csv('C:/Users/koyounghun/Desktop/찌르레기/Loan_data/y_smote.csv')

# ADASYN

In [7]:
start = time.time()

adasyn = ADASYN(random_state = 101)

X_over_ada, y_over_ada = adasyn.fit_resample(X_train, y_train)


end = time.time()


print(f"{end - start:.5f} sec")

3.30800 sec


a , b = X_over_ada, y_over_ada


a = pd.DataFrame(a)
b = pd.DataFrame(b)

a.to_csv('C:/Users/koyounghun/Desktop/찌르레기/Loan_data/x_adasyn.csv')
b.to_csv('C:/Users/koyounghun/Desktop/찌르레기/Loan_data/y_adasyn.csv')

# distribution smote generate using colab enviroment

# modeling with random over sampling
# logit

In [115]:
from sklearn.linear_model import LogisticRegression

start = time.time()

lr_clf = LogisticRegression()
get_model_train_eval(lr_clf , ftr_train = X_ros, ftr_test = X_test, tgt_train = y_ros, tgt_test = y_val )

end = time.time()


print(f"{end - start:.5f} sec")

오차행렬
[[39758 26534]
 [ 5281  4027]]
정확도: 0.579167, 정밀도 0.131769, 재현율 0.432639, AUC 0.516190,G_MEAN 0.509383
2.58000 sec


# Decision Tree

In [114]:
start = time.time()

tree = DecisionTreeClassifier()


get_model_train_eval(tree , ftr_train = X_ros, ftr_test = X_val, tgt_train = y_ros, tgt_test = y_val )


end = time.time()


print(f"{end - start:.5f} sec")

오차행렬
[[58854  7438]
 [ 1732  7576]]
정확도: 0.878704, 정밀도 0.504596, 재현율 0.813924, AUC 0.850861,G_MEAN 0.850059
141.32800 sec


# random forest

In [116]:
start = time.time()

rlf =RandomForestClassifier(n_estimators=100, 
                               random_state=101, 
                               min_samples_leaf=2, 
                               criterion='entropy',
                           n_jobs = -1)

get_model_train_eval(rlf , ftr_train = X_ros, ftr_test = X_val, tgt_train = y_ros, tgt_test = y_val)

end = time.time()


print(f"{end - start:.5f} sec")

오차행렬
[[60459  5833]
 [ 2120  7188]]
정확도: 0.894802, 정밀도 0.552031, 재현율 0.772239, AUC 0.842125,G_MEAN 0.839220
32.81000 sec


# XGB

In [122]:
from xgboost import XGBClassifier

start = time.time()

xgb = XGBClassifier(learning_rate=0.1,
                    n_estimators=1000,
                    use_label_encoder=False,
                    random_state=2021)

get_model_train_eval(xgb , ftr_train = X_ros, ftr_test = X_val.values, tgt_train = y_ros, tgt_test = y_val.values )

end = time.time()

print(f"{end - start:.5f} sec")

오차행렬
[[51655 14637]
 [ 2220  7088]]
정확도: 0.777024, 정밀도 0.326260, 재현율 0.761495, AUC 0.770350,G_MEAN 0.770299
263.78959 sec


# LGB

In [117]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

start = time.time()

lgb = LGBMClassifier(n_estimators= 400)
get_model_train_eval(lgb , ftr_train = X_ros, ftr_test = X_val, tgt_train = y_ros, tgt_test = y_val )


end = time.time()

print(f"{end - start:.5f} sec")

오차행렬
[[58268  8024]
 [ 1775  7533]]
정확도: 0.870384, 정밀도 0.484219, 재현율 0.809304, AUC 0.844132,G_MEAN 0.843413
The time of execution of above program is : 12.880002737045288


# LDA

In [118]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import time

start = time.time()

lda = LinearDiscriminantAnalysis()
get_model_train_eval(lda , ftr_train = X_smote, ftr_test = X_val, tgt_train = y_smote, tgt_test = y_val )


end = time.time()

print(f"{end - start:.5f} sec")

오차행렬
[[33796 32496]
 [ 4259  5049]]
정확도: 0.513823, 정밀도 0.134479, 재현율 0.542437, AUC 0.526121,G_MEAN 0.525868
time : 0.8100001811981201


# QDA

In [119]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

start = time.time()

qda = QuadraticDiscriminantAnalysis()
get_model_train_eval(qda , ftr_train = X_smote, ftr_test = X_val, tgt_train = y_smote, tgt_test = y_val )

end = time.time()

print(f"{end - start:.5f} sec")

오차행렬
[[29605 36687]
 [ 3409  5899]]
정확도: 0.469630, 정밀도 0.138520, 재현율 0.633756, AUC 0.540170,G_MEAN 0.532002
time : 0.3469994068145752


# MLP

In [120]:
from sklearn.neural_network import MLPClassifier

start = time.time()

nn = MLPClassifier()
get_model_train_eval(nn , ftr_train = X_smote, ftr_test = X_val, tgt_train = y_smote, tgt_test = y_val )


end = time.time()

print(f"{end - start:.5f} sec")

오차행렬
[[66292     0]
 [ 9308     0]]
정확도: 0.876878, 정밀도 0.000000, 재현율 0.000000, AUC 0.500000,G_MEAN 0.000000
time : 710.8001878261566


# SVM

In [ ]:
from sklearn import svm

start = time.time()

svm = svm.SVC()

get_model_train_eval(svm , ftr_train = X_ros, ftr_test = X_val, tgt_train = y_ros, tgt_test = y_val )

end = time.time()

print(f"{end - start:.5f} sec")

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)

start = time.time()



get_model_train_eval(knn , ftr_train = X_ros, ftr_test = X_val, tgt_train = y_ros, tgt_test = y_val )

end = time.time()

print(f"{end - start:.5f} sec")


# modeling with no oversampling

In [ ]:
start = time.time()

print('logit')

get_model_train_eval(lr_clf , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()


start = time.time()

print('decision tree')

get_model_train_eval(tree , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()

start = time.time()

print('random forest')


get_model_train_eval(rlf , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()



In [ ]:
start = time.time()

print('xgb')

get_model_train_eval(xgb , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()


start = time.time()

print('lgb')

get_model_train_eval(lgb , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()

start = time.time()

print('lda')


get_model_train_eval(lda , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()

In [ ]:
start = time.time()

print('qda')

get_model_train_eval(qda , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()


start = time.time()

print('nn')

get_model_train_eval(nn , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()

start = time.time()

print('svm')


get_model_train_eval(svm , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()


start = time.time()

print('knn')


get_model_train_eval(knn , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()

# modeling with adasyn

In [ ]:
start = time.time()

print('logit')

get_model_train_eval(lr_clf , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()


start = time.time()

print('decision tree')

get_model_train_eval(tree , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()

start = time.time()

print('random forest')


get_model_train_eval(rlf , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()



In [ ]:
start = time.time()

print('xgb')

get_model_train_eval(xgb , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()


start = time.time()

print('lgb')

get_model_train_eval(lgb , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()

start = time.time()

print('lda')


get_model_train_eval(lda , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()

In [ ]:
start = time.time()

print('qda')

get_model_train_eval(qda , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()


start = time.time()

print('nn')

get_model_train_eval(nn , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()

start = time.time()

print('svm')


get_model_train_eval(svm , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()


start = time.time()

print('knn')


get_model_train_eval(knn , ftr_train = X_train, ftr_test = X_val, tgt_train = y_train, tgt_test = y_val )

end = time.time()